# Goal

to fetch the events from the calendar and edit them as a timebox given prompt and then write them back.

That way we can give this as a tool to the schedular agent as well, so it can change a single event, but also reschedule the day as a whole if needed.

In [1]:
from datetime import datetime, timedelta
work_start_hour = 8 # TODO: move to settings
is_before_work_start = datetime.now().time() < datetime.now().replace(hour=work_start_hour, minute=0, second=0, microsecond=0).time()
# is_before_work_start = True
today = datetime.now().date()
day_to_plan = today if is_before_work_start else today + timedelta(days=1)
day_to_plan

datetime.date(2026, 2, 4)

# now pull events from calendar for that date as a timebox object

```mermaid

flowchart TB
  node_1("CalendarEvent")

```

In [2]:
from dataclasses import dataclass
from datetime import datetime, date as date_type, time as time_type, timedelta, timezone
from typing import Any, Iterable
from zoneinfo import ZoneInfo

from dateutil import parser as date_parser

from fateforger.agents.schedular.models.calendar import CalendarEvent, EventType
from fateforger.agents.timeboxing.timebox import Timebox


from autogen_ext.tools.mcp import McpWorkbench, StreamableHttpServerParams

from fateforger.core.config import settings


# TODO: wrao this in a nice function

def calendar_workbench() -> McpWorkbench:

    return McpWorkbench(
        StreamableHttpServerParams(url=settings.mcp_calendar_server_url, timeout=10.0)
    )
workbench = calendar_workbench()

# query the workbench
result = await workbench.call_tool(
    "list-events", # TODO: what are all the args we can pass here?
    arguments={
        "calendarId": "primary",
        "timeMin": "2026-01-27T00:00:00Z",
        "timeMax": "2026-01-28T00:00:00Z",
        "singleEvents": True,
        "orderBy": "startTime",
    },
)

raw_json_string = result.result[0].content

# def get_timebox_for_day(day: date_type, timezone_str: str) -> Timebox:

In [3]:
result.result[0].content

'{"events":[{"id":"30287b8c333b44c18b3bfbd52f357d4f","summary":"morning review","start":{"dateTime":"2026-01-27T13:00:00+01:00","timeZone":"Europe/Amsterdam"},"end":{"dateTime":"2026-01-27T13:15:00+01:00","timeZone":"Europe/Amsterdam"},"status":"confirmed","htmlLink":"https://www.google.com/calendar/event?eid=MzAyODdiOGMzMzNiNDRjMThiM2JmYmQ1MmYzNTdkNGYgaHVnby5ldmVyc0Bt","created":"2026-01-26T21:07:30.000Z","updated":"2026-01-27T10:54:23.636Z","colorId":"10","creator":{"email":"hugo.evers@gmail.com","self":true},"organizer":{"email":"hugo.evers@gmail.com","self":true},"iCalUID":"30287b8c333b44c18b3bfbd52f357d4f@google.com","sequence":1,"reminders":{"useDefault":true},"eventType":"default","guestsCanModify":true,"calendarId":"primary","accountId":"normal"}],"totalCount":1}'

In [4]:
from fateforger.adapters.calendar.models import GCalEventsResponse

gcal_events = GCalEventsResponse.model_validate_json(result.result[0].content)

In [5]:
gcal_events

GCalEventsResponse(events=[GCalEvent(id='30287b8c333b44c18b3bfbd52f357d4f', summary='morning review', start=GCalEventDateTime(date_time='2026-01-27T13:00:00+01:00', date=None, time_zone='Europe/Amsterdam'), end=GCalEventDateTime(date_time='2026-01-27T13:15:00+01:00', date=None, time_zone='Europe/Amsterdam'), status='confirmed', html_link='https://www.google.com/calendar/event?eid=MzAyODdiOGMzMzNiNDRjMThiM2JmYmQ1MmYzNTdkNGYgaHVnby5ldmVyc0Bt', created='2026-01-26T21:07:30.000Z', updated='2026-01-27T10:54:23.636Z', creator=GCalPerson(email='hugo.evers@gmail.com', self_=True), organizer=GCalPerson(email='hugo.evers@gmail.com', self_=True), ical_uid='30287b8c333b44c18b3bfbd52f357d4f@google.com', sequence=1, reminders=GCalReminders(use_default=True, overrides=None), event_type='default', guests_can_modify=True, calendar_id='primary', account_id='normal', colorId='10')], total_count=1)

# convert that to Timebox

what is the goal here?
we want to give the agent a tool so it can pull the timebox for a given day.

we want to take that json object and turn it into a pydantic object.

# TODO: link to code

now we want to have a model that we use for timeboxing, so we convert it into a timebox object, l

In [6]:
# import from src/fateforger/adapters/calendar/models.py


In [7]:
from fateforger.agents.schedular.models.calendar import CalendarEvent



In [8]:
# now let the agent do it
# agent needs to know which day to fetch and the timezone, it can default to users.info for the timezone.
# and assume the user wants to plan for the next work day if before work start hour, unless the user specifies otherwise.


# Inspecting the timebox object

does it have all the properties/aliases to match the calendar event objects so we can mix and match directly?
Does the timebox object have the serialisation methods to inject it into the agents' context?

Proposal: Productionize ObjectPatcher and Replace Timeboxing Agent Patching
1) Where the code should live (module ownership)
Primary module:
patcher.py
Reason: this is timeboxing‑specific behavior (LLM patching of list‑based “event specs”), and keeps the scheduling domain clean.

Supporting types (if needed):
patching.py
Reason: holds PatchResult, ErrorInfo, and shared schemas to avoid circular imports.

Public entry point:
__init__.py should export ObjectPatcher (or renamed TimeboxingPatcher) so other agents can import it.

2) Replace/modify existing flow
Current likely patching code:
patching.py (open in tabs earlier).
Plan: either replace with the new patcher or move it into patcher.py and keep old file as a thin wrapper (if any call sites rely on it).

Replace where patching is invoked:
Likely in:

agent.py
flow.py or flow_graph.py
We will route the patching step to the new AutoGen tool‑calling patcher using build_autogen_chat_client.

3) The patcher API in production
Production class (renaming optional):
ObjectPatcher[T] with:

run(instruction: str, max_attempts: int) -> PatchResult[T]
restart_from("initial" | "last_success")
PatchResult includes:
ok: bool
result: T (always present)
current_artifact: T
applied: list[dict]
error: str | None
summary: str | None
error_info: ErrorInfo | None (category + message + failing op)
This mirrors the notebook logic.

4) AutoGen integration
Use AssistantAgent + FunctionTool(strict=True) with tool schema derived from PatchRequest.
Use build_autogen_chat_client("timeboxing_patcher", model=<env override>).
The tool returns a typed PatchResult (not raw dict).
Agent will be used in the timeboxing stage, not as a separate long‑lived agent.
5) Docs changes
README updates

README.md
Add a Status section (Implemented/Documented/Tested/etc).
Add “Patching” subsection describing tool‑calling patcher and error repair.
Docs (MkDocs)

Add page in docs/ (e.g., patching.md).
Update mkdocs.yml nav to include it.
AGENTS.md

AGENTS.md
Add note that patching uses AutoGen tool‑calling + strict schema; no deterministic NLU.
Mention the new patcher module and its ownership boundary.
6) Tests
Add tests in tests/timeboxing/ (or nearest existing timeboxing tests):

Success path: valid patch that adds/updates items.
Failure path: invalid patch (bad index) → error_info.category == "invalid_index".
Repair path: patch_of_patch when last patch exists.
Missing last patch: patch_of_patch when no last patch → patch_of_patch_missing.
These tests should not hit the live LLM; use a mocked AutoGen model client or a fake agent with deterministic tool calls.

7) Migration plan
Extract notebook code into patcher.py.
Wire patcher into timeboxing flow (agent.py / flow.py).
Update docs + AGENTS.
Add tests + run targeted pytest subset.
Questions I need answered before coding
Exact replacement target: Which file/function is the patching stage today?
Naming: keep ObjectPatcher, or rename to TimeboxingPatcher?
What should be the model in production? (Probably your timeboxing event spec model).
If this proposal is aligned, say “approved” and answer the three questions, and I’ll draft the formal ticket and start the implementation.

# now apply json patches

for this part we must determine what to prompt the model, so its going to generate json patches, but in which model? it needs to use the minimized schema, it will need to know about the current timebox object, it needs instructions, the inputs.

initially we dont want it to use issue json patches to create a timebox, instead it should print in markdown a rough draft of the timebox, which we can then convert to json patches to create the timebox object.
the question is if we assume the user agreees with the rought draft whether we can create the first real timebox.

In [9]:
# the timeboxing agent should have as it memory the timebox object, the inputs, and the last user message

# stage 3: rough draft of timebox

## TODO: 
* make sure the constraints and inputs are passed correctly to the rough draft agent, this means also injecting the current timebox events as fixed events.
* parametrize the planning rules and draft algorithm
* validate the markdown output and the planning trace output
* make sure we can insert this is toon input into the stage 4 timebox json patching agent

```xml
<SystemPrompt>
  <Role>Rough-Draft Day Planner Agent</Role>

  <Objective>
    Generate a bird’s-eye, glanceable rough draft of a day plan from user constraints and inputs.
    The output is meant to be refined in later stages; prioritize clarity, order, and approximate durations.
  </Objective>

  <Inputs>
    <Constraints>
      Fixed events with exact start/end (sleep, meetings, travel).
      Day start time (e.g., work starts 08:00).
      Preferred block sizes (e.g., DW=2h, half-DW=1h, quarter-DW=30m).
      Preferences (e.g., gym early, stimulant window ~4h).
    </Constraints>
    <Tasks>
      DailyOneThing (single most important outcome).
      Supporting tasks (meal prep, chores, fun/creative time).
      Optional constraints about stimulant timing, energy, or sequencing.
    </Tasks>
  </Inputs>

  <PlanningRules>
    <Rule id="R1">Honor all fixed-time constraints exactly; do not overlap events.</Rule>
    <Rule id="R2">Place DailyOneThing into 2–3 Deep Work blocks early-to-midday unless constraints prevent it.</Rule>
    <Rule id="R3">Default DW blocks to the user’s standard duration; allow 1h or 30m “partial DW” if specified.</Rule>
    <Rule id="R4">Insert buffers where risk is high: after DW (10–15m), after gym (30–45m reset), after meals (10–20m).</Rule>
    <Rule id="R5">Keep Shallow Work ≤30% of waking hours; cluster it later in the day when possible.</Rule>
    <Rule id="R6">Protect recovery and fun/creative blocks as first-class events; do not label them “optional” unless user did.</Rule>
    <Rule id="R7">If stimulants are mentioned: align demanding DW within the effective window; avoid back-to-back dosing unless user insists.</Rule>
    <Rule id="R8">Prefer simple sequencing over precision. This is a rough draft for later refinement.</Rule>
  </PlanningRules>

  <DraftAlgorithm>
    <Step>Lock fixed anchors (sleep, immovable meetings, hard start time).</Step>
    <Step>Place Morning Routine immediately after sleep end (15–45m, per user preference).</Step>
    <Step>Schedule DW blocks for DailyOneThing: hardest block first, then execution/polish blocks.</Step>
    <Step>Place gym in the user’s preferred slot (e.g., early) and add a reset buffer after.</Step>
    <Step>Place a fun/unstructured block (1–2h) if requested, ideally between stimulant windows or after a major block.</Step>
    <Step>Place shallow work (chores/admin) later; keep it bounded.</Step>
    <Step>Place meal prep + dinner in the evening; then creative time if requested.</Step>
    <Step>End with shutdown ritual + reading in bed + sleep anchor.</Step>
  </DraftAlgorithm>

  <OutputSpec>
    <PrimaryOutput format="markdown">
      <![CDATA[
      Produce a short, glanceable summary with:
      - A few headings (Night/Morning/Midday/Afternoon/Evening) OR emojis (optional).
      - ONE line per major chunk (not per micro-event).
      - Each line: **Label** — rough duration(or start-end if fixed) (e.g., "Deep Work (primary) — 2h").
      - Do not include exact timestamps unless the user explicitly asked for times.
      - Keep total output under ~12–15 lines.
      ]]>
    </PrimaryOutput>

    <PlanningTrace format="xml">
      <TraceRules>
        <Rule>Do NOT include chain-of-thought or detailed internal reasoning.</Rule>
        <Rule>Include only: assumptions, placements, durations, and checks.</Rule>
      </TraceRules>
      <TraceTemplate>
        <![CDATA[
        <PlanTrace>
          <Assumptions>
            <Assumption id="A1">...</Assumption>
          </Assumptions>
          <Placements>
            <Placement type="DW" label="DailyOneThing" duration="PT2H" rationale="hardest first"/>
            <Placement type="H" label="Gym" duration="PT1H30M" rationale="early preference"/>
          </Placements>
          <Checks>
            <Check name="NoOverlaps" status="pass"/>
            <Check name="ShallowWorkRatio" status="pass|warn" value="..."/>
            <Check name="DWCount" status="pass|warn" value="..."/>
          </Checks>
        </PlanTrace>
        ]]>
      </TraceTemplate>
    </PlanningTrace>

    <Tone>
      Crisp, minimal, readable at a glance. No motivational speeches.
    </Tone>
  </OutputSpec>

  <FailureModes>
    <Mode id="F1">Missing fixed anchors: infer a reasonable sleep block only if user provided sleep duration + bedtime; otherwise ask one question.</Mode>
    <Mode id="F2">Too many tasks: prioritize DailyOneThing and cap DW at 3 blocks; push the rest to shallow work or mark as overflow.</Mode>
    <Mode id="F3">Timing conflict: preserve fixed events, then reflow flexible blocks; keep buffers if possible.</Mode>
  </FailureModes>
</SystemPrompt>

```

In [10]:
from llama_index.tools.artifact_editor import (
    ArtifactEditorToolSpec,
    ArtifactMemoryBlock,
)

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os
try:
    load_dotenv()
except AssertionError:
    load_dotenv(dotenv_path=".env")


token = os.getenv("OPENROUTER_API_KEY")
open_router_base_url = os.getenv("OPENROUTER_BASE_URL")
client = OpenAI(api_key=token, base_url=open_router_base_url)
model="google/gemini-3-flash-preview"

In [ ]:
from pydantic import BaseModel
from typing import Literal, Optional

class PizzaOrder(BaseModel):
    size: Literal["small", "medium", "large", "extra-large"]
    toppings: list[str]
    address: str


schema = PizzaOrder.model_json_schema()

prompt = lambda order: f"""
You are an expert pizza ordering agent. You need to order a pizza for the user based on their preferences.
you respond with json and list the size, toppings and address to deliver to.
Here is the order:
{order}
"""

order = "I want a large pizza with pepperoni and mushrooms, deliver to 123 Main St."

response = client.responses.create(
  model=model,
  input=[{"role": "user", "content": prompt(order)}],
  text={
    "format": {
      "type": "json_schema",
      "name": "shopping_list",
      "strict": True,
      "schema": schema
    }
  }
)
pizza_order = PizzaOrder.model_validate_json(response.output_text)
pizza_order

PizzaOrder(size='large', toppings=['pepperoni', 'mushrooms'], address='123 Main St.')

In [ ]:
Style = Literal["napoletana", "new-york", "sicilian", "detroit", "roman"]
Size = Literal["s", "m", "l", "xl"]


class Pizza(BaseModel):
    style: Style
    toppings: str
    size: Size
    quantity: int = Field(ge=1)


class PizzaOrder(BaseModel):
    pizzas: list[Pizza] = Field(default_factory=list)



In [ ]:
import json
from typing import Annotated, Generic, Literal, Optional, Type, TypeVar, Union, get_args, get_origin

from pydantic import BaseModel, Field, create_model, model_validator
from llama_index.tools.artifact_editor import ArtifactEditorToolSpec
from llama_index.tools.artifact_editor.base import JsonPatch
from jinja2 import Template

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool

from fateforger.llm import build_autogen_chat_client



T = TypeVar("T", bound=BaseModel)


ErrorCategory = Literal[
    "invalid_path",
    "invalid_index",
    "invalid_enum",
    "missing_field",
    "schema_validation",
    "tool_call_failure",
    "patch_of_patch_missing",
    "unknown",
]


class ErrorInfo(BaseModel):
    category: ErrorCategory
    message: str
    op: dict | None = None
    op_index: int | None = None


class PatchResult(BaseModel, Generic[T]):
    ok: bool
    result: T
    current_artifact: T
    applied: list[dict] = Field(default_factory=list)
    error: str | None = None
    summary: str | None = None
    error_info: ErrorInfo | None = None


def _unwrap_optional(annotation: Type) -> Type:
    origin = get_origin(annotation)
    if origin is Union:
        args = [arg for arg in get_args(annotation) if arg is not type(None)]
        if len(args) == 1:
            return args[0]
    return annotation


def _infer_list_field(model_cls: Type[BaseModel]) -> str:
    # General-purpose: expect exactly one list field unless overridden.
    fields = getattr(model_cls, "model_fields", None) or getattr(model_cls, "__fields__", {})
    list_fields = []
    for name, field in fields.items():
        annotation = getattr(field, "annotation", None) or getattr(field, "outer_type_", None)
        annotation = _unwrap_optional(annotation)
        if get_origin(annotation) is list:
            list_fields.append(name)
    if len(list_fields) != 1:
        raise ValueError(f"Expected exactly one list field, found: {list_fields}")
    return list_fields[0]


def _infer_item_model(model_cls: Type[T], list_field: str) -> Type:
    field = (getattr(model_cls, "model_fields", None) or getattr(model_cls, "__fields__", {}))[list_field]
    annotation = getattr(field, "annotation", None) or getattr(field, "outer_type_", None)
    annotation = _unwrap_optional(annotation)
    args = get_args(annotation)
    if not args:
        raise ValueError(f"List field {list_field} is missing item type")
    return args[0]


def make_patch_schema(item_model: Type) -> Type[BaseModel]:
    # Typed patch schema: add/replace values MUST be valid item objects.
    add_op = create_model(
        "AddOp",
        op=(Literal["add"], "add"),
        path=(str, Field(..., description="JSON pointer path")),
        value=(item_model, Field(..., description="Item to add")),
    )
    replace_op = create_model(
        "ReplaceOp",
        op=(Literal["replace"], "replace"),
        path=(str, Field(..., description="JSON pointer path")),
        value=(item_model, Field(..., description="Full item replacement")),
    )
    remove_op = create_model(
        "RemoveOp",
        op=(Literal["remove"], "remove"),
        path=(str, Field(..., description="JSON pointer path")),
    )
    move_op = create_model(
        "MoveOp",
        op=(Literal["move"], "move"),
        from_path=(str, Field(..., description="Source path (use from_path)")),
        path=(str, Field(..., description="Target path")),
    )
    copy_op = create_model(
        "CopyOp",
        op=(Literal["copy"], "copy"),
        from_path=(str, Field(..., description="Source path (use from_path)")),
        path=(str, Field(..., description="Target path")),
    )
    patch_op = Annotated[
        Union[add_op, replace_op, remove_op, move_op, copy_op],
        Field(discriminator="op"),
    ]
    patch_plan = create_model(
        "PatchPlan",
        operations=(list[patch_op], Field(..., description="RFC6902 ops")),
    )
    return patch_plan


def make_request_model(patch_plan: Type[BaseModel]) -> Type[BaseModel]:
    class PatchRequest(BaseModel):
        mode: Literal["new", "patch_of_patch"]
        patch: patch_plan | JsonPatch

        @model_validator(mode="after")
        def _validate_mode(self):
            if self.mode == "new" and not isinstance(self.patch, patch_plan):
                raise ValueError("mode=new requires a full patch plan")
            if self.mode == "patch_of_patch" and not isinstance(self.patch, JsonPatch):
                raise ValueError("mode=patch_of_patch requires a JsonPatch")
            return self

    return PatchRequest


def _classify_error(exc: Exception, *, op: dict | None = None, op_index: int | None = None) -> ErrorInfo:
    message = str(exc)
    lower = message.lower()

    if "no previous patch" in lower:
        category = "patch_of_patch_missing"
    elif isinstance(exc, IndexError) or "out of range" in lower:
        category = "invalid_index"
    elif isinstance(exc, KeyError) or "invalid field" in lower or "cannot access nested field" in lower or "path" in lower:
        category = "invalid_path"
    elif "field required" in lower:
        category = "missing_field"
    elif "input should be" in lower or "literal" in lower or "enum" in lower:
        category = "invalid_enum"
    elif "patch resulted in invalid" in lower or "validation" in lower:
        category = "schema_validation"
    else:
        category = "unknown"

    return ErrorInfo(category=category, message=message, op=op, op_index=op_index)


class ObjectPatcher(Generic[T]):
    # General-purpose patcher for "model with one list field" using AutoGen tool calling.
    def __init__(
        self,
        model_cls: Type[T],
        *,
        model_client,
        schema_by_alias: bool = False,
        initial_state: BaseModel | dict | None = None,
        list_field: str | None = None,
    ) -> None:
        self.model_cls = model_cls
        self.schema_by_alias = schema_by_alias
        self.list_field = list_field or _infer_list_field(model_cls)
        self.item_model = _infer_item_model(model_cls, self.list_field)
        self.patch_plan = make_patch_schema(self.item_model)
        self.PatchRequest = make_request_model(self.patch_plan)
        self.patch_request_schema = self.PatchRequest.model_json_schema(by_alias=self.schema_by_alias)
        self.ResultModel = PatchResult[model_cls]
        self.spec = ArtifactEditorToolSpec(model_cls)
        self._set_initial_state(initial_state)
        self._initial_snapshot = self._artifact()
        self._last_patch: BaseModel | None = None
        self._last_success: BaseModel | None = None

        self._agent = AssistantAgent(
            name="PatchAgent",
            model_client=model_client,
            tools=[self._build_tool()],
            max_tool_iterations=1,
            tool_call_summary_format="{result}",
            system_message=PATCH_SYSTEM_MESSAGE,
        )
        self._summarizer = AssistantAgent(
            name="PatchSummarizer",
            model_client=model_client,
            system_message=SUMMARY_SYSTEM_MESSAGE,
        )

    def _build_tool(self) -> FunctionTool:
        async def apply_patch(request: self.PatchRequest) -> PatchResult[T]:  # type: ignore[name-defined]
            return self._handle_request(request)

        return FunctionTool(
            apply_patch,
            description="Apply a patch request to the current artifact.",
            strict=True,
        )

    def _set_initial_state(self, initial_state: BaseModel | dict | None) -> None:
        if initial_state is None:
            data = {self.list_field: []}
        elif isinstance(initial_state, BaseModel):
            data = initial_state.model_dump()
        else:
            data = dict(initial_state)
        if self.list_field not in data:
            data[self.list_field] = []

        # Allow invalid inputs by constructing items without validation; patcher fixes them.
        if isinstance(self.item_model, type) and issubclass(self.item_model, BaseModel):
            items = []
            for item in data[self.list_field]:
                if isinstance(item, BaseModel):
                    items.append(item)
                else:
                    items.append(self.item_model.model_construct(**dict(item)))
            data[self.list_field] = items

        try:
            self.spec.current_artifact = self.model_cls.model_validate(data)
        except Exception:
            self.spec.current_artifact = self.model_cls.model_construct(**data)

    def _artifact(self) -> dict:
        return self.spec.get_current_artifact() or {self.list_field: []}

    def _handle_request(self, request: BaseModel) -> BaseModel:
        patch = request.patch
        if request.mode == "patch_of_patch":
            try:
                patch = self._apply_patch_to_patch(request.patch)
            except Exception as exc:
                error_info = _classify_error(exc)
                return self.ResultModel.model_validate(
                    {
                        "ok": False,
                        "result": self.model_cls.model_construct(**self._artifact()),
                        "current_artifact": self.model_cls.model_construct(**self._artifact()),
                        "applied": [],
                        "error": str(exc),
                        "summary": None,
                        "error_info": error_info,
                    }
                )
        self._last_patch = patch
        return self._apply_patch_sequential(patch)

    def _apply_patch_to_patch(self, patch_delta: JsonPatch) -> BaseModel:
        if self._last_patch is None:
            raise ValueError("No previous patch to repair.")
        editor = ArtifactEditorToolSpec(self.patch_plan)
        editor.current_artifact = self._last_patch
        updated = editor.apply_patch(patch_delta)
        return self.patch_plan.model_validate(updated)

    def _apply_patch_sequential(self, patch: BaseModel) -> PatchResult[T]:
        # Apply op-by-op so we can recover at the last successful patch.
        applied: list[dict] = []
        for op_index, op in enumerate(patch.operations):
            op_dict = op.model_dump(exclude_none=True)
            try:
                self.spec.apply_patch(JsonPatch.model_validate({"operations": [op_dict]}))
                applied.append(op_dict)
            except Exception as exc:
                error_info = _classify_error(exc, op=op_dict, op_index=op_index)
                return self.ResultModel.model_validate(
                    {
                        "ok": False,
                        "result": self.model_cls.model_construct(**self._artifact()),
                        "current_artifact": self.model_cls.model_construct(**self._artifact()),
                        "applied": applied,
                        "error": str(exc),
                        "summary": None,
                        "error_info": error_info,
                    }
                )
        result_model = self.model_cls.model_validate(self._artifact())
        self._last_success = result_model
        return self.ResultModel.model_validate(
            {
                "ok": True,
                "result": result_model,
                "current_artifact": result_model,
                "applied": applied,
                "error": None,
                "summary": None,
                "error_info": None,
            }
        )

    def _render_prompt(self, instruction: str, error: str | None, error_info: ErrorInfo | None) -> str:
        template = Template(PATCH_TEMPLATE)
        return template.render(
            list_field=self.list_field,
            artifact=json.dumps(self._artifact(), indent=2),
            instruction=instruction,
            item_schema=json.dumps(self.item_model.model_json_schema(by_alias=self.schema_by_alias), indent=2)
            if hasattr(self.item_model, "model_json_schema")
            else json.dumps({}, indent=2),
            patch_schema=json.dumps(self.patch_plan.model_json_schema(), indent=2),
            request_schema=json.dumps(self.patch_request_schema, indent=2),
            last_patch=json.dumps(self._last_patch.model_dump(), indent=2) if self._last_patch else "null",
            error_block=(f"Previous error: {error}" if error else ""),
            error_info=json.dumps(error_info.model_dump(), indent=2) if error_info else "null",
        )

    async def run(self, instruction: str, max_attempts: int = 3) -> PatchResult[T]:
        last_error: str | None = None
        last_error_info: ErrorInfo | None = None
        for _ in range(max_attempts):
            prompt = self._render_prompt(instruction, last_error, last_error_info)
            response = await self._agent.on_messages([TextMessage(content=prompt, source="user")], CancellationToken())
            try:
                outcome = self.ResultModel.model_validate_json(response.chat_message.content)
            except Exception as exc:
                last_error = f"Invalid tool result: {exc}"
                continue
            if outcome.ok:
                return outcome
            last_error = outcome.error
            last_error_info = outcome.error_info

        summary = await self._summarize_failure(last_error or "unknown", last_error_info)
        return self.ResultModel.model_validate(
            {
                "ok": False,
                "result": self.model_cls.model_construct(**self._artifact()),
                "current_artifact": self.model_cls.model_construct(**self._artifact()),
                "applied": [],
                "error": last_error,
                "summary": summary,
                "error_info": last_error_info,
            }
        )

    async def _summarize_failure(self, error: str, error_info: ErrorInfo | None) -> str:
        summary_prompt = Template(SUMMARY_TEMPLATE).render(
            error=error,
            error_info=json.dumps(error_info.model_dump(), indent=2) if error_info else "null",
            artifact=json.dumps(self._artifact(), indent=2),
            last_patch=json.dumps(self._last_patch.model_dump(), indent=2) if self._last_patch else "null",
        )
        response = await self._summarizer.on_messages([TextMessage(content=summary_prompt, source="user")], CancellationToken())
        return response.chat_message.content.strip()

    def restart_from(self, state: Literal["initial", "last_success"]) -> None:
        if state == "last_success" and self._last_success is not None:
            self._set_initial_state(self._last_success)
        else:
            self._set_initial_state(self._initial_snapshot)

In [6]:
model = globals().get("model") or "google/gemini-3-flash-preview"

PATCH_SYSTEM_MESSAGE = """
You are a patching agent. Your job is to call the apply_patch tool with a PatchRequest.
Do not answer with prose. Always call the tool.
""".strip()

PATCH_TEMPLATE = """
You are editing a list in a JSON artifact.
Goal: make the list match the user's instruction.

Rules:
- Use ONLY the apply_patch tool.
- Choose mode = "new" for a fresh patch plan.
- Choose mode = "patch_of_patch" to repair the last patch plan.
- Operations are applied sequentially; indices refer to the current state.
- For add/replace, value MUST be a full item matching the item schema.
- Allowed ops: add, replace, remove, move, copy.
- Use from_path (not from) for move/copy.
- Prefer small, minimal patches.
- Replace whole list items (use path like /{{list_field}}/0), not subfields.

List field: /{{list_field}}

Current artifact:
{{artifact}}

User instruction:
{{instruction}}

Item schema:
{{item_schema}}

Patch schema:
{{patch_schema}}

PatchRequest schema:
{{request_schema}}

Last patch (for patch_of_patch mode):
{{last_patch}}

Error info (structured):
{{error_info}}

{{error_block}}
""".strip()

SUMMARY_SYSTEM_MESSAGE = """
You are a concise failure summarizer. Respond with 2-3 bullets.
""".strip()

SUMMARY_TEMPLATE = """
Summarize the patch failure in 2-3 bullets and suggest next steps.

Error: {{error}}
Error info: {{error_info}}
Current artifact: {{artifact}}
Last patch: {{last_patch}}
""".strip()


# Patch walkthrough (AutoGen + ArtifactEditor)

**Purpose:** Patch an *incorrect* list-based object into the desired state using **AutoGen tool calling** + **typed JSON Patch**.

## Steps (mapped to code)
1. **Infer list + item types**: `_infer_list_field` + `_infer_item_model` find the list field and its item type.  
2. **Typed patch schema**: `make_patch_schema(item_model)` forces `add/replace.value` to be valid items.  
3. **PatchRequest schema**: `make_request_schema` lets the agent choose **new** patch or **patch_of_patch**.  
4. **Tool call**: `ObjectPatcher._build_tool` exposes `apply_patch(request)` as an AutoGen tool (strict schema).  
5. **Sequential apply**: `_apply_patch_sequential` applies op-by-op for recoverability.  
6. **Repair loop**: `run` retries; on failure, summarize and allow restart options.

## Trade-offs
- **Pros:** schema enforcement *before* patching, AutoGen tool loop, recoverable failures.
- **Cons:** assumes one list field per model; sequential patching is slower but safer for index shifts.


# Mermaid diagrams

```mermaid
flowchart TD
  A[User instruction] --> B[ObjectPatcher.run]
  B --> C[_render_prompt]
  C --> D[AutoGen AssistantAgent]
  D --> E[apply_patch tool]
  E --> F[ArtifactEditorToolSpec.apply_patch]
  F --> G[Updated artifact]
```

```mermaid
sequenceDiagram
  participant U as User
  participant P as ObjectPatcher
  participant A as AutoGen Agent
  participant T as apply_patch tool
  participant E as ArtifactEditor

  U->>P: instruction + initial_state
  P->>A: prompt (artifact + schemas)
  A->>T: PatchRequest(mode=new|patch_of_patch)
  T->>E: apply patch ops sequentially
  E-->>T: updated artifact or error
  T-->>A: PatchOutcome (JSON)
  A-->>P: tool summary (JSON)
```


# TODOs
- [ ] Allow models with multiple list fields (explicit list_field required).
- [ ] Add a small "patch preview" mode (no apply, just diff).
- [ ] Expand repair prompts with structured error categories.


In [2]:
wrong_pizza_order = {
    "pizzas": [
        {"style": "napolitaon", "toppings": "ananas", "size": "m", "quantity": 1}
    ]
}

model_client = build_autogen_chat_client("patch_agent", model=model)
patcher = ObjectPatcher(PizzaOrder, model_client=model_client, initial_state=wrong_pizza_order)

instruction = (
    "Make it a napoletana, size l, toppings pepperoni and mushrooms, quantity 2. "
    "Add a second pizza: sicilian, size s, toppings olives, quantity 1."
)

result = await patcher.run(instruction, max_attempts=3)
result


NameError: name 'build_autogen_chat_client' is not defined

In [ ]:
result

PatchResult[PizzaOrder](ok=True, result=PizzaOrder(pizzas=[Pizza(style='napoletana', toppings='pepperoni and mushrooms', size='l', quantity=2), Pizza(style='sicilian', toppings='olives', size='s', quantity=1)]), current_artifact=PizzaOrder(pizzas=[Pizza(style='napoletana', toppings='pepperoni and mushrooms', size='l', quantity=2), Pizza(style='sicilian', toppings='olives', size='s', quantity=1)]), applied=[{'op': 'replace', 'path': '/pizzas/0', 'value': {'style': 'napoletana', 'toppings': 'pepperoni and mushrooms', 'size': 'l', 'quantity': 2}}, {'op': 'add', 'path': '/pizzas/1', 'value': {'style': 'sicilian', 'toppings': 'olives', 'size': 's', 'quantity': 1}}], error=None, summary=None, error_info=None)

In [ ]:
planning_session = 


def make_planning_session(start_time: datetime=None, end_time: datetime=None,duration:TimeDelta=None) -> CalendarEvent:
    time_details = {}
    return CalendarEvent(
        summary="Planning Session for Timeboxing",
        **time_details
    )

In [ ]:
from fateforger.agents.timeboxing.timebox import Timebox,CalendarEvent

# Lightweight Timebox Generation Models

## Design goals
1. **Token-efficient** — short aliases for every field so the LLM generates less JSON
2. **Discriminated timing** — 4 anchoring variants instead of 3 optional fields + boolean
3. **Domain ops** — typed operations instead of generic JSON Patch
4. **Convertible** — deterministic `resolve_times()` → `GCalEvent` for the MCP tool

## Alias map
| Model field | Alias | Meaning |
|---|---|---|
| `TBEvent.n` | — | name / summary |
| `TBEvent.d` | — | description |
| `TBEvent.t` | — | event type (`DW`, `H`, `R`, …) |
| `TBEvent.p` | — | placement / timing |
| `Timing.a` | — | anchor kind (`ap`, `bn`, `fs`, `fw`) |
| `dur` | — | duration (ISO 8601) |
| `st` | — | start time (HH:MM) |
| `et` | — | end time (HH:MM) |

## Anchor types
- `ap` — **after previous**: starts when previous ends (default, most events)
- `bn` — **before next**: ends when next event starts
- `fs` — **fixed start**: pinned start + duration
- `fw` — **fixed window**: pinned start + end

## Token comparison (5-event timebox)
```
CalendarEvent (full):  ~180 tokens per event → ~900 total
TBEvent (aliased):     ~40 tokens per event  → ~200 total
                       ~4.5× fewer tokens
```

In [1]:
"""Lightweight timebox models for token-efficient LLM generation."""

from __future__ import annotations

from datetime import date as date_type, datetime, time, timedelta
from enum import Enum
from typing import Annotated, Literal, Union

from isodate import parse_duration as _parse_dur
from pydantic import BaseModel, ConfigDict, Field, field_validator, model_validator


# ── EventType (reuse existing short codes) ────────────────────────────────
# We mirror the production EventType values but as a plain str enum
# so we avoid SQLAlchemy/ChoiceEnum machinery in the generation path.

class ET(str, Enum):
    """Event type — compact codes for LLM generation."""
    M  = "M"    # meeting
    C  = "C"    # commute
    DW = "DW"   # deep work
    SW = "SW"   # shallow work
    PR = "PR"   # plan & review
    H  = "H"    # habit / routine
    R  = "R"    # regeneration (meals, sleep, rest)
    BU = "BU"   # buffer
    BG = "BG"   # background (must have fixed timing)


# ── Time anchoring (discriminated union) ──────────────────────────────────

class AfterPrev(BaseModel):
    """Starts immediately after the previous event ends. Default."""
    model_config = ConfigDict(extra="forbid")
    a: Literal["ap"] = "ap"
    dur: timedelta = Field(..., description="Duration (ISO 8601, e.g. PT30M)")

    _parse = field_validator("dur", mode="before")(lambda cls, v: _parse_dur(v) if isinstance(v, str) else v)


class BeforeNext(BaseModel):
    """Ends immediately when the next event starts."""
    model_config = ConfigDict(extra="forbid")
    a: Literal["bn"] = "bn"
    dur: timedelta = Field(..., description="Duration (ISO 8601)")

    _parse = field_validator("dur", mode="before")(lambda cls, v: _parse_dur(v) if isinstance(v, str) else v)


class FixedStart(BaseModel):
    """Pinned to a specific start time."""
    model_config = ConfigDict(extra="forbid")
    a: Literal["fs"] = "fs"
    st: time = Field(..., description="Start time (HH:MM)")
    dur: timedelta = Field(..., description="Duration (ISO 8601)")

    _parse_t = field_validator("st", mode="before")(lambda cls, v: time.fromisoformat(v) if isinstance(v, str) else v)
    _parse_d = field_validator("dur", mode="before")(lambda cls, v: _parse_dur(v) if isinstance(v, str) else v)


class FixedWindow(BaseModel):
    """Pinned start and end — for meetings, background events, etc."""
    model_config = ConfigDict(extra="forbid")
    a: Literal["fw"] = "fw"
    st: time = Field(..., description="Start time (HH:MM)")
    et: time = Field(..., description="End time (HH:MM)")

    _parse_st = field_validator("st", mode="before")(lambda cls, v: time.fromisoformat(v) if isinstance(v, str) else v)
    _parse_et = field_validator("et", mode="before")(lambda cls, v: time.fromisoformat(v) if isinstance(v, str) else v)


Timing = Annotated[
    Union[AfterPrev, BeforeNext, FixedStart, FixedWindow],
    Field(discriminator="a"),
]


# ── TBEvent (the generation-time event model) ────────────────────────────

class TBEvent(BaseModel):
    """A single timeboxed event — minimal fields for LLM generation."""
    model_config = ConfigDict(extra="forbid")

    n: str = Field(..., description="Event name / summary")
    d: str = Field("", description="Short description")
    t: ET = Field(..., description="Event type code")
    p: Timing = Field(..., description="Time placement")

    @model_validator(mode="after")
    def bg_needs_fixed(self) -> "TBEvent":
        """Background events must have a fixed window or fixed start."""
        if self.t == ET.BG and self.p.a not in ("fs", "fw"):
            raise ValueError("Background events (BG) require fixed_start or fixed_window timing")
        return self


# ── TBPlan (the generation-time timebox) ──────────────────────────────────

class TBPlan(BaseModel):
    """A day's timebox plan — lightweight container for LLM generation."""
    model_config = ConfigDict(extra="forbid")

    events: list[TBEvent] = Field(default_factory=list)
    date: date_type = Field(default_factory=date_type.today)
    tz: str = Field(default="Europe/Amsterdam", description="IANA timezone")

    @model_validator(mode="after")
    def chain_must_be_anchored(self) -> "TBPlan":
        """At least one non-BG event must have a fixed time to anchor the chain."""
        chain = [e for e in self.events if e.t != ET.BG]
        if chain and not any(e.p.a in ("fs", "fw") for e in chain):
            raise ValueError(
                "Event chain needs at least one fixed_start or fixed_window anchor"
            )
        return self

    def resolve_times(self) -> list[dict]:
        """
        Deterministically compute concrete start/end for every event.
        Returns list of dicts: [{n, d, t, start_time, end_time, duration}, ...]
        """
        planning_date = self.date
        resolved: list[dict] = []

        # ── Forward pass: resolve after_previous and fixed_start/fixed_window ──
        last_end_dt: datetime | None = None
        for i, ev in enumerate(self.events):
            r = {"n": ev.n, "d": ev.d, "t": ev.t.value, "index": i}
            p = ev.p

            if p.a == "ap":  # after_previous
                if last_end_dt is None:
                    raise ValueError(f"Event '{ev.n}' (after_previous) has no preceding event to anchor to")
                start_dt = last_end_dt
                end_dt = start_dt + p.dur
                r.update(start_time=start_dt.time(), end_time=end_dt.time(), duration=p.dur)

            elif p.a == "fs":  # fixed_start
                start_dt = datetime.combine(planning_date, p.st)
                end_dt = start_dt + p.dur
                r.update(start_time=p.st, end_time=end_dt.time(), duration=p.dur)

            elif p.a == "fw":  # fixed_window
                start_dt = datetime.combine(planning_date, p.st)
                end_dt = datetime.combine(planning_date, p.et)
                r.update(start_time=p.st, end_time=p.et, duration=end_dt - start_dt)

            elif p.a == "bn":  # before_next — placeholder, resolved in backward pass
                r.update(duration=p.dur, _pending="bn")
                resolved.append(r)
                continue  # don't update last_end_dt yet

            last_end_dt = datetime.combine(planning_date, r["end_time"])
            resolved.append(r)

        # ── Backward pass: resolve before_next ──
        next_start_dt: datetime | None = None
        for r in reversed(resolved):
            if r.get("_pending") == "bn":
                if next_start_dt is None:
                    raise ValueError(f"Event '{r['n']}' (before_next) has no following event to anchor to")
                end_dt = next_start_dt
                start_dt = end_dt - r["duration"]
                r.update(start_time=start_dt.time(), end_time=end_dt.time())
                del r["_pending"]
            if "start_time" in r:
                next_start_dt = datetime.combine(planning_date, r["start_time"])

        # ── Overlap check (non-BG only) ──
        chain = [r for r in resolved if r["t"] != "BG"]
        for a, b in zip(chain, chain[1:]):
            a_end = datetime.combine(planning_date, a["end_time"])
            b_start = datetime.combine(planning_date, b["start_time"])
            if a_end > b_start:
                raise ValueError(f"Overlap: '{a['n']}' ends {a['end_time']} but '{b['n']}' starts {b['start_time']}")

        return resolved


# ── Quick schema preview ──────────────────────────────────────────────────
print("=== TBEvent JSON Schema ===")
import json
print(json.dumps(TBEvent.model_json_schema(), indent=2))

=== TBEvent JSON Schema ===
{
  "$defs": {
    "AfterPrev": {
      "additionalProperties": false,
      "description": "Starts immediately after the previous event ends. Default.",
      "properties": {
        "a": {
          "const": "ap",
          "default": "ap",
          "title": "A",
          "type": "string"
        },
        "dur": {
          "description": "Duration (ISO 8601, e.g. PT30M)",
          "format": "duration",
          "title": "Dur",
          "type": "string"
        }
      },
      "required": [
        "dur"
      ],
      "title": "AfterPrev",
      "type": "object"
    },
    "BeforeNext": {
      "additionalProperties": false,
      "description": "Ends immediately when the next event starts.",
      "properties": {
        "a": {
          "const": "bn",
          "default": "bn",
          "title": "A",
          "type": "string"
        },
        "dur": {
          "description": "Duration (ISO 8601)",
          "format": "duration",
          "

In [5]:
"""Domain-specific operations for timebox patching."""


# ── Operations ────────────────────────────────────────────────────────────
# Each op is a discriminated union member.  The LLM picks the op type
# and gets a typed schema — no generic JSON Patch paths or `value: Any`.

class AddEvents(BaseModel):
    """Add one or more events. `after` = insert position (None → append)."""
    model_config = ConfigDict(extra="forbid")
    op: Literal["ae"] = "ae"
    events: list[TBEvent] = Field(..., min_length=1)
    after: int | None = Field(None, description="Insert after this index (None=append)")


class RemoveEvent(BaseModel):
    """Remove an event by index."""
    model_config = ConfigDict(extra="forbid")
    op: Literal["re"] = "re"
    i: int = Field(..., description="Index of event to remove")


class UpdateEvent(BaseModel):
    """Update specific fields on an existing event. Only set fields are changed."""
    model_config = ConfigDict(extra="forbid")
    op: Literal["ue"] = "ue"
    i: int = Field(..., description="Index of event to update")
    n: str | None = Field(None, description="New name")
    d: str | None = Field(None, description="New description")
    t: ET | None = Field(None, description="New event type")
    p: Timing | None = Field(None, description="New time placement")


class MoveEvent(BaseModel):
    """Move an event to a different position in the ordered list."""
    model_config = ConfigDict(extra="forbid")
    op: Literal["me"] = "me"
    fr: int = Field(..., description="From index")
    to: int = Field(..., description="To index")


class ReplaceAll(BaseModel):
    """Replace the entire event list (initial generation or full rebuild)."""
    model_config = ConfigDict(extra="forbid")
    op: Literal["ra"] = "ra"
    events: list[TBEvent] = Field(..., min_length=1)


TBOp = Annotated[
    Union[AddEvents, RemoveEvent, UpdateEvent, MoveEvent, ReplaceAll],
    Field(discriminator="op"),
]


class TBPatch(BaseModel):
    """A batch of typed operations to apply to a TBPlan."""
    model_config = ConfigDict(extra="forbid")
    ops: list[TBOp] = Field(..., min_length=1)


# ── Patch applicator ─────────────────────────────────────────────────────

def apply_tb_ops(plan: TBPlan, patch: TBPatch) -> TBPlan:
    """Apply domain operations sequentially, return a new validated TBPlan."""
    events = list(plan.events)  # mutable copy

    for op in patch.ops:
        match op.op:
            case "ae":  # add_events
                if op.after is not None:
                    for offset, ev in enumerate(op.events):
                        events.insert(op.after + 1 + offset, ev)
                else:
                    events.extend(op.events)

            case "re":  # remove_event
                if op.i < 0 or op.i >= len(events):
                    raise IndexError(f"remove: index {op.i} out of range (0..{len(events)-1})")
                events.pop(op.i)

            case "ue":  # update_event
                if op.i < 0 or op.i >= len(events):
                    raise IndexError(f"update: index {op.i} out of range (0..{len(events)-1})")
                current = events[op.i]
                # Build merged dict and re-validate to preserve discriminated unions
                merged = current.model_dump()
                updates = {k: v for k, v in [
                    ("n", op.n), ("d", op.d), ("t", op.t), ("p", op.p),
                ] if v is not None}
                # For Timing (p), pass the model directly so Pydantic re-validates
                if "p" in updates and isinstance(updates["p"], BaseModel):
                    updates["p"] = updates["p"].model_dump()
                if "t" in updates and isinstance(updates["t"], ET):
                    updates["t"] = updates["t"].value
                merged.update(updates)
                events[op.i] = TBEvent.model_validate(merged)

            case "me":  # move_event
                if op.fr < 0 or op.fr >= len(events):
                    raise IndexError(f"move: from_index {op.fr} out of range")
                ev = events.pop(op.fr)
                to = min(op.to, len(events))
                events.insert(to, ev)

            case "ra":  # replace_all
                events = list(op.events)

    return TBPlan(events=events, date=plan.date, tz=plan.tz)


print("=== TBPatch JSON Schema ===")
print(json.dumps(TBPatch.model_json_schema(), indent=2))

=== TBPatch JSON Schema ===
{
  "$defs": {
    "AddEvents": {
      "additionalProperties": false,
      "description": "Add one or more events. `after` = insert position (None \u2192 append).",
      "properties": {
        "op": {
          "const": "ae",
          "default": "ae",
          "title": "Op",
          "type": "string"
        },
        "events": {
          "items": {
            "$ref": "#/$defs/TBEvent"
          },
          "minItems": 1,
          "title": "Events",
          "type": "array"
        },
        "after": {
          "anyOf": [
            {
              "type": "integer"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "description": "Insert after this index (None=append)",
          "title": "After"
        }
      },
      "required": [
        "events"
      ],
      "title": "AddEvents",
      "type": "object"
    },
    "AfterPrev": {
      "additionalProperties": false,

In [3]:
"""Conversion: TBPlan → GCalEvent list for the MCP calendar tool."""

from zoneinfo import ZoneInfo
from fateforger.adapters.calendar.models import GCalEvent, GCalEventDateTime

# Map ET codes → production EventType color_ids for Google Calendar
_ET_COLOR_MAP: dict[str, str] = {
    "M": "6", "C": "4", "DW": "9", "SW": "8",
    "PR": "10", "H": "7", "R": "2", "BU": "5", "BG": "1",
}


def tb_plan_to_gcal_events(plan: TBPlan) -> list[GCalEvent]:
    """
    Resolve times and convert a TBPlan into GCalEvent objects
    ready for the Google Calendar MCP `create-event` tool.
    """
    resolved = plan.resolve_times()
    tz = ZoneInfo(plan.tz)
    gcal_events: list[GCalEvent] = []

    for r in resolved:
        start_dt = datetime.combine(plan.date, r["start_time"], tzinfo=tz)
        end_dt = datetime.combine(plan.date, r["end_time"], tzinfo=tz)

        gcal_events.append(
            GCalEvent.model_construct(
                id="",  # will be assigned by Google Calendar
                summary=r["n"],
                start=GCalEventDateTime(
                    date_time=start_dt.isoformat(),
                    time_zone=plan.tz,
                ),
                end=GCalEventDateTime(
                    date_time=end_dt.isoformat(),
                    time_zone=plan.tz,
                ),
                status="confirmed",
                # colorId is passed separately via the MCP tool call
            )
        )

    return gcal_events


# ── Test the full pipeline ────────────────────────────────────────────────

# 1) Build a sample plan using compact aliases
sample_plan = TBPlan(
    date=date_type(2026, 2, 7),
    tz="Europe/Amsterdam",
    events=[
        TBEvent(n="Morning routine", d="Shower, coffee", t=ET.H,
                p=FixedStart(st=time(7, 0), dur=timedelta(minutes=30))),
        TBEvent(n="Deep work: thesis", d="Chapter 3 draft", t=ET.DW,
                p=AfterPrev(dur=timedelta(hours=2))),
        TBEvent(n="Coffee break", d="", t=ET.BU,
                p=AfterPrev(dur=timedelta(minutes=15))),
        TBEvent(n="Standup", d="Team sync", t=ET.M,
                p=FixedWindow(st=time(10, 0), et=time(10, 15))),
        TBEvent(n="Deep work: thesis", d="Polish + references", t=ET.DW,
                p=AfterPrev(dur=timedelta(hours=1, minutes=30))),
        TBEvent(n="Lunch", d="", t=ET.R,
                p=AfterPrev(dur=timedelta(minutes=45))),
    ],
)

# 2) Resolve times
resolved = sample_plan.resolve_times()
print("=== Resolved schedule ===")
for r in resolved:
    print(f"  {r['start_time'].strftime('%H:%M')}–{r['end_time'].strftime('%H:%M')}  {r['n']} ({r['t']})")

# 3) Convert to GCalEvent
gcal_events = tb_plan_to_gcal_events(sample_plan)
print(f"\n=== {len(gcal_events)} GCalEvent objects ready for MCP ===")
for ge in gcal_events:
    print(f"  {ge.summary}: {ge.start.date_time} → {ge.end.date_time}")

# 4) Show token savings: what the LLM actually generates
raw_json = sample_plan.model_dump_json(indent=2)
print(f"\n=== TBPlan JSON size: {len(raw_json)} chars ===")
print(raw_json[:500], "..." if len(raw_json) > 500 else "")

=== Resolved schedule ===
  07:00–07:30  Morning routine (H)
  07:30–09:30  Deep work: thesis (DW)
  09:30–09:45  Coffee break (BU)
  10:00–10:15  Standup (M)
  10:15–11:45  Deep work: thesis (DW)
  11:45–12:30  Lunch (R)

=== 6 GCalEvent objects ready for MCP ===
  Morning routine: 2026-02-07T07:00:00+01:00 → 2026-02-07T07:30:00+01:00
  Deep work: thesis: 2026-02-07T07:30:00+01:00 → 2026-02-07T09:30:00+01:00
  Coffee break: 2026-02-07T09:30:00+01:00 → 2026-02-07T09:45:00+01:00
  Standup: 2026-02-07T10:00:00+01:00 → 2026-02-07T10:15:00+01:00
  Deep work: thesis: 2026-02-07T10:15:00+01:00 → 2026-02-07T11:45:00+01:00
  Lunch: 2026-02-07T11:45:00+01:00 → 2026-02-07T12:30:00+01:00

=== TBPlan JSON size: 992 chars ===
{
  "events": [
    {
      "n": "Morning routine",
      "d": "Shower, coffee",
      "t": "H",
      "p": {
        "a": "fs",
        "st": "07:00:00",
        "dur": "PT30M"
      }
    },
    {
      "n": "Deep work: thesis",
      "d": "Chapter 3 draft",
      "t": "DW",

In [6]:
"""Test domain ops: patch the sample plan with realistic user requests."""

# Scenario: "Move the standup earlier, extend deep work, add gym before lunch"
patch = TBPatch(ops=[
    # Update standup to 09:45-10:00
    UpdateEvent(i=3, p=FixedWindow(st=time(9, 45), et=time(10, 0))),
    # Extend second deep work to 2 hours
    UpdateEvent(i=4, p=AfterPrev(dur=timedelta(hours=2))),
    # Add gym before lunch
    AddEvents(
        after=4,
        events=[TBEvent(n="Gym", d="Strength training", t=ET.H,
                        p=AfterPrev(dur=timedelta(hours=1)))],
    ),
])

patched = apply_tb_ops(sample_plan, patch)
resolved_patched = patched.resolve_times()

print("=== Patched schedule ===")
for r in resolved_patched:
    print(f"  {r['start_time'].strftime('%H:%M')}–{r['end_time'].strftime('%H:%M')}  {r['n']} ({r['t']})")

# Show the patch JSON (what LLM would output)
print(f"\n=== Patch JSON ({len(patch.model_dump_json())} chars) ===")
print(patch.model_dump_json(indent=2))

=== Patched schedule ===
  07:00–07:30  Morning routine (H)
  07:30–09:30  Deep work: thesis (DW)
  09:30–09:45  Coffee break (BU)
  09:45–10:00  Standup (M)
  10:00–12:00  Deep work: thesis (DW)
  12:00–13:00  Gym (H)
  13:00–13:45  Lunch (R)

=== Patch JSON (279 chars) ===
{
  "ops": [
    {
      "op": "ue",
      "i": 3,
      "n": null,
      "d": null,
      "t": null,
      "p": {
        "a": "fw",
        "st": "09:45:00",
        "et": "10:00:00"
      }
    },
    {
      "op": "ue",
      "i": 4,
      "n": null,
      "d": null,
      "t": null,
      "p": {
        "a": "ap",
        "dur": "PT2H"
      }
    },
    {
      "op": "ae",
      "events": [
        {
          "n": "Gym",
          "d": "Strength training",
          "t": "H",
          "p": {
            "a": "ap",
            "dur": "PT1H"
          }
        }
      ],
      "after": 4
    }
  ]
}


# End-to-End Pipeline: GCal MCP → TBPlan → Patch → Submit

## Flow
```
GCal MCP list-events → GCalEventsResponse → TBPlan (immovables as fw)
    ↓
LLM generates TBPatch (domain ops) against TBPlan
    ↓
apply_tb_ops → patched TBPlan
    ↓
resolve_times() → concrete start/end for every event
    ↓
tb_plan_to_gcal_create_args() → list of MCP create-event argument dicts
    ↓
diff with original → create/update/delete ops
    ↓
BatchCalendarSubmitter.apply() via MCP
```

## Key mappings
| GCal MCP field | TBPlan field | Direction |
|---|---|---|
| `summary` | `TBEvent.n` | both ways |
| `description` | `TBEvent.d` | both ways |
| `start.dateTime` | resolved `start_time` | receive → `fw`; submit → ISO8601 |
| `end.dateTime` | resolved `end_time` | receive → `fw`; submit → ISO8601 |
| `colorId` | `_ET_COLOR_MAP[TBEvent.t]` | submit only |
| `id` / `eventId` | tracked for diff | receive + submit |

In [13]:
"""
Step 1: GCal MCP → TBPlan
    Fetch events from Google Calendar via MCP and convert to our lightweight TBPlan.
    Existing calendar events become fixed-window (fw) anchors.

Step 2: TBPlan → GCal MCP create-event args
    Convert resolved TBPlan events back into the exact dict shape
    that the MCP `create-event` / `update-event` tools expect.
"""

import base64
import hashlib
from dataclasses import dataclass

from autogen_ext.tools.mcp import McpWorkbench, StreamableHttpServerParams
from dateutil import parser as date_parser
from fateforger.adapters.calendar.models import GCalEventsResponse
from fateforger.core.config import settings


# ── GCal MCP → TBPlan ────────────────────────────────────────────────────

def _gcal_color_to_et(color_id: str | None) -> ET:
    """Best-effort reverse mapping: GCal colorId → ET code."""
    _reverse = {v: k for k, v in _ET_COLOR_MAP.items()}
    if color_id and color_id in _reverse:
        return ET(_reverse[color_id])
    return ET.M  # default: treat unknown calendar events as meetings


def gcal_response_to_tb_plan(
    resp: GCalEventsResponse,
    *,
    plan_date: date_type,
    tz_name: str = "Europe/Amsterdam",
) -> TBPlan:
    """
    Convert a GCalEventsResponse (from MCP list-events) into a TBPlan.
    
    All existing calendar events become FixedWindow anchors since
    they already have concrete start/end times.
    """
    tz = ZoneInfo(tz_name)
    events: list[TBEvent] = []

    for ge in resp.events:
        # Skip all-day events (no dateTime)
        if not ge.start.date_time or not ge.end.date_time:
            continue
        
        # Skip cancelled
        if ge.status and ge.status.lower() == "cancelled":
            continue

        start_dt = date_parser.isoparse(ge.start.date_time).astimezone(tz)
        end_dt = date_parser.isoparse(ge.end.date_time).astimezone(tz)

        # Skip events not on our planning date
        if start_dt.date() != plan_date:
            continue

        # Detect event type from colorId if available
        et = _gcal_color_to_et(getattr(ge, "colorId", None) or getattr(ge, "color_id", None))

        events.append(TBEvent(
            n=ge.summary or "Busy",
            d="",  # GCal description is often long HTML; skip for generation context
            t=et,
            p=FixedWindow(st=start_dt.time(), et=end_dt.time()),
        ))

    # Sort by start time
    events.sort(key=lambda e: e.p.st if hasattr(e.p, "st") else time(0, 0))

    return TBPlan(events=events, date=plan_date, tz=tz_name)


# ── TBPlan → MCP create-event args ───────────────────────────────────────

def _base32hex_id(seed: str, *, prefix: str = "fftb", max_len: int = 64) -> str:
    """Deterministic event ID for owned timebox events.
    
    GCal event IDs must only contain lowercase a-v and 0-9 (base32hex).
    """
    digest = hashlib.sha1(seed.encode("utf-8")).digest()
    token = base64.b32hexencode(digest).decode("ascii").lower().rstrip("=")
    return (prefix + token)[:max_len]


@dataclass
class MCPCalendarOp:
    """A single MCP calendar operation (create / update / delete)."""
    tool_name: str  # "create-event" | "update-event" | "delete-event"
    arguments: dict


def tb_plan_to_mcp_ops(
    plan: TBPlan,
    *,
    original_event_ids: dict[int, str] | None = None,
    calendar_id: str = "primary",
) -> list[MCPCalendarOp]:
    """
    Convert a resolved TBPlan into MCP tool call arguments.
    
    Args:
        plan: The TBPlan with events to submit.
        original_event_ids: {event_index: gcal_event_id} for events that
            already exist in GCal (so we can update instead of create).
        calendar_id: Target calendar.
    
    Returns:
        List of MCPCalendarOp with tool_name + arguments dicts.
    """
    resolved = plan.resolve_times()
    tz = ZoneInfo(plan.tz)
    original_event_ids = original_event_ids or {}
    ops: list[MCPCalendarOp] = []

    for r in resolved:
        idx = r["index"]
        start_dt = datetime.combine(plan.date, r["start_time"], tzinfo=tz)
        end_dt = datetime.combine(plan.date, r["end_time"], tzinfo=tz)
        color_id = _ET_COLOR_MAP.get(r["t"], "0")

        args = {
            "calendarId": calendar_id,
            "summary": r["n"],
            "description": r["d"] or "",
            "start": start_dt.isoformat(),
            "end": end_dt.isoformat(),
            "timeZone": plan.tz,
            "colorId": color_id,
        }

        if idx in original_event_ids:
            # This event already exists in GCal → update
            args["eventId"] = original_event_ids[idx]
            ops.append(MCPCalendarOp(tool_name="update-event", arguments=args))
        else:
            # New event → create with deterministic ID
            seed = f"{plan.date}|{r['n']}|{r['start_time']}|{idx}"
            args["eventId"] = _base32hex_id(seed)
            ops.append(MCPCalendarOp(tool_name="create-event", arguments=args))

    return ops


def diff_tb_plans(
    before: TBPlan,
    after: TBPlan,
    *,
    event_id_map: dict[int, str],
    calendar_id: str = "primary",
) -> list[MCPCalendarOp]:
    """
    Diff two TBPlans and return the minimal set of MCP ops.
    
    Args:
        before: Original plan (from GCal).
        after: Patched plan.
        event_id_map: {before_index: gcal_event_id} for events in `before`.
        calendar_id: Target calendar.
    
    Returns:
        create/update/delete MCPCalendarOps.
    """
    before_resolved = {r["index"]: r for r in before.resolve_times()}
    after_resolved = after.resolve_times()
    tz = ZoneInfo(after.tz)
    ops: list[MCPCalendarOp] = []

    after_indices = set()

    for r in after_resolved:
        idx = r["index"]
        after_indices.add(idx)
        start_dt = datetime.combine(after.date, r["start_time"], tzinfo=tz)
        end_dt = datetime.combine(after.date, r["end_time"], tzinfo=tz)
        color_id = _ET_COLOR_MAP.get(r["t"], "0")

        args = {
            "calendarId": calendar_id,
            "summary": r["n"],
            "description": r["d"] or "",
            "start": start_dt.isoformat(),
            "end": end_dt.isoformat(),
            "timeZone": after.tz,
            "colorId": color_id,
        }

        if idx in event_id_map:
            # Check if actually changed
            old = before_resolved.get(idx)
            if old and (old["n"] != r["n"] or old["start_time"] != r["start_time"]
                        or old["end_time"] != r["end_time"] or old.get("d") != r.get("d")):
                args["eventId"] = event_id_map[idx]
                ops.append(MCPCalendarOp(tool_name="update-event", arguments=args))
        else:
            seed = f"{after.date}|{r['n']}|{r['start_time']}|{idx}"
            args["eventId"] = _base32hex_id(seed)
            ops.append(MCPCalendarOp(tool_name="create-event", arguments=args))

    # Deletes: events in before that are not in after
    for before_idx, gcal_id in event_id_map.items():
        if before_idx not in after_indices:
            ops.append(MCPCalendarOp(
                tool_name="delete-event",
                arguments={"calendarId": calendar_id, "eventId": gcal_id},
            ))

    return ops


# ── MCP Batch Submitter ──────────────────────────────────────────────────

class TBSubmitter:
    """Submit TBPlan operations to Google Calendar via MCP."""

    def __init__(self, *, server_url: str | None = None, timeout_s: float = 10.0):
        url = server_url or settings.mcp_calendar_server_url
        self._workbench = McpWorkbench(
            StreamableHttpServerParams(url=url, timeout=timeout_s)
        )

    async def apply_ops(self, ops: list[MCPCalendarOp]) -> list[dict]:
        """Execute MCP ops sequentially and return results."""
        results = []
        for op in ops:
            result = await self._workbench.call_tool(op.tool_name, arguments=op.arguments)
            results.append({
                "tool": op.tool_name,
                "event_id": op.arguments.get("eventId"),
                "summary": op.arguments.get("summary"),
                "ok": not getattr(result, "is_error", False),
                "content": getattr(result, "content", str(result)),
            })
        return results

    async def submit_plan(
        self,
        plan: TBPlan,
        *,
        original_event_ids: dict[int, str] | None = None,
        calendar_id: str = "primary",
    ) -> list[dict]:
        """Resolve times, build ops, and submit to GCal."""
        ops = tb_plan_to_mcp_ops(
            plan,
            original_event_ids=original_event_ids,
            calendar_id=calendar_id,
        )
        return await self.apply_ops(ops)


print("Pipeline loaded: gcal_response_to_tb_plan, tb_plan_to_mcp_ops, diff_tb_plans, TBSubmitter")

Pipeline loaded: gcal_response_to_tb_plan, tb_plan_to_mcp_ops, diff_tb_plans, TBSubmitter


In [9]:
"""
Full pipeline test (offline — no MCP server needed).
Uses simulated GCal events + LLM-generated plan, then shows MCP ops.
"""

# ── Simulate: pretend these 2 events came from GCal list-events ──────────
from fateforger.adapters.calendar.models import GCalEventsResponse, GCalEvent, GCalEventDateTime

fake_gcal_response = GCalEventsResponse(
    events=[
        GCalEvent(
            id="abc123",
            summary="Team standup",
            start=GCalEventDateTime(dateTime="2026-02-08T10:00:00+01:00", timeZone="Europe/Amsterdam"),
            end=GCalEventDateTime(dateTime="2026-02-08T10:15:00+01:00", timeZone="Europe/Amsterdam"),
            status="confirmed",
        ),
        GCalEvent(
            id="def456",
            summary="Lunch with Sarah",
            start=GCalEventDateTime(dateTime="2026-02-08T12:30:00+01:00", timeZone="Europe/Amsterdam"),
            end=GCalEventDateTime(dateTime="2026-02-08T13:30:00+01:00", timeZone="Europe/Amsterdam"),
            status="confirmed",
        ),
    ],
    totalCount=2,
)

plan_date = date_type(2026, 2, 8)

# ── Step 1: GCal → TBPlan (immovables as fixed windows) ──────────────────
baseline = gcal_response_to_tb_plan(fake_gcal_response, plan_date=plan_date)
print("=== Baseline from GCal (immovables) ===")
for i, ev in enumerate(baseline.events):
    print(f"  [{i}] {ev.n} ({ev.t.value}) — {ev.p}")

# Track which GCal event IDs map to summaries (for after-patching lookup)
gcal_id_by_summary = {ge.summary: ge.id for ge in fake_gcal_response.events}
print(f"\nGCal IDs: {gcal_id_by_summary}")

# ── Step 2: LLM generates full plan via replace_all ──────────────────────
#  Initial generation = ReplaceAll: the LLM builds the whole day,
#  keeping immovables as FixedWindow and filling gaps with new events.
llm_patch = TBPatch(ops=[
    ReplaceAll(events=[
        # Morning block
        TBEvent(n="Morning routine", d="Shower + coffee", t=ET.H,
                p=FixedStart(st=time(7, 0), dur=timedelta(minutes=30))),
        TBEvent(n="Deep work: thesis", d="Chapter 3", t=ET.DW,
                p=AfterPrev(dur=timedelta(hours=2))),
        TBEvent(n="Coffee break", d="", t=ET.BU,
                p=AfterPrev(dur=timedelta(minutes=15))),
        # Immovable: standup (LLM preserves it as-is)
        TBEvent(n="Team standup", d="", t=ET.M,
                p=FixedWindow(st=time(10, 0), et=time(10, 15))),
        # Fill gap after standup
        TBEvent(n="Deep work: thesis", d="Polish draft", t=ET.DW,
                p=AfterPrev(dur=timedelta(hours=1, minutes=30))),
        # Immovable: lunch (LLM preserves it as-is)
        TBEvent(n="Lunch with Sarah", d="", t=ET.M,
                p=FixedWindow(st=time(12, 30), et=time(13, 30))),
        # Afternoon
        TBEvent(n="Shallow work", d="Emails + admin", t=ET.SW,
                p=AfterPrev(dur=timedelta(minutes=45))),
        TBEvent(n="Gym", d="", t=ET.H,
                p=AfterPrev(dur=timedelta(hours=1))),
    ]),
])

# ── Step 3: Apply patch ──────────────────────────────────────────────────
patched_plan = apply_tb_ops(baseline, llm_patch)
resolved = patched_plan.resolve_times()

print("\n=== Patched schedule ===")
for r in resolved:
    print(f"  {r['start_time'].strftime('%H:%M')}–{r['end_time'].strftime('%H:%M')}  {r['n']} ({r['t']})")

# ── Step 4: Map GCal IDs to patched indices ──────────────────────────────
# Match by summary to find which patched events are existing GCal events
original_event_ids: dict[int, str] = {}
for r in resolved:
    if r["n"] in gcal_id_by_summary:
        original_event_ids[r["index"]] = gcal_id_by_summary[r["n"]]
print(f"\nMatched GCal IDs: {original_event_ids}")

# ── Step 5: Generate MCP ops ─────────────────────────────────────────────
ops = tb_plan_to_mcp_ops(
    patched_plan,
    original_event_ids=original_event_ids,
    calendar_id="primary",
)

print(f"\n=== {len(ops)} MCP operations ===")
for op in ops:
    verb = op.tool_name.replace("-event", "").upper()
    summary = op.arguments.get("summary", "")
    eid = op.arguments.get("eventId", "")[:20]
    start = op.arguments.get("start", "")
    print(f"  {verb:8s} {summary:25s} id={eid}  start={start}")

# ── Show what the LLM actually generated (token count) ───────────────────
patch_json = llm_patch.model_dump_json()
print(f"\n=== LLM output size: {len(patch_json)} chars ===")

=== Baseline from GCal (immovables) ===
  [0] Team standup (M) — a='fw' st=datetime.time(10, 0) et=datetime.time(10, 15)
  [1] Lunch with Sarah (M) — a='fw' st=datetime.time(12, 30) et=datetime.time(13, 30)

GCal IDs: {'Team standup': 'abc123', 'Lunch with Sarah': 'def456'}

=== Patched schedule ===
  07:00–07:30  Morning routine (H)
  07:30–09:30  Deep work: thesis (DW)
  09:30–09:45  Coffee break (BU)
  10:00–10:15  Team standup (M)
  10:15–11:45  Deep work: thesis (DW)
  12:30–13:30  Lunch with Sarah (M)
  13:30–14:15  Shallow work (SW)
  14:15–15:15  Gym (H)

Matched GCal IDs: {3: 'abc123', 5: 'def456'}

=== 8 MCP operations ===
  CREATE   Morning routine           id=fftb_j73av1ltk1pmge3  start=2026-02-08T07:00:00+01:00
  CREATE   Deep work: thesis         id=fftb_9uqg08rcpmb55r1  start=2026-02-08T07:30:00+01:00
  CREATE   Coffee break              id=fftb_02itvpp3vo75aop  start=2026-02-08T09:30:00+01:00
  UPDATE   Team standup              id=abc123  start=2026-02-08T10:00:00+01:

## 🔴 Live MCP Pipeline Test

**End-to-end**: GCal MCP `list-events` → `TBPlan` → patch → `create-event` / `update-event` back to GCal.

Uses the real calendar MCP at `http://localhost:3000`.

In [11]:
"""
Live MCP test — Step 1: Fetch real events from GCal via MCP.
"""
import asyncio
import json
from datetime import date as date_type, datetime, time, timedelta
from zoneinfo import ZoneInfo

from autogen_ext.tools.mcp import McpWorkbench, StreamableHttpServerParams
from fateforger.adapters.calendar.models import GCalEventsResponse

MCP_URL = "http://localhost:3000"
TZ = "Europe/Amsterdam"

# ── Use tomorrow so we can freely create/delete without messing up today ──
plan_date = date_type(2026, 2, 8)

async def fetch_events(target_date: date_type) -> str:
    """Fetch events from GCal MCP for a specific date, returns raw JSON string."""
    wb = McpWorkbench(StreamableHttpServerParams(url=MCP_URL, timeout=15))
    async with wb:
        result = await wb.call_tool(
            "list-events",
            arguments={
                "calendarId": "primary",
                "timeMin": datetime.combine(target_date, time(0, 0), tzinfo=ZoneInfo(TZ)).isoformat(),
                "timeMax": datetime.combine(target_date + timedelta(days=1), time(0, 0), tzinfo=ZoneInfo(TZ)).isoformat(),
            },
        )
    # ToolResult → .result is a list of TextResultContent
    for part in result.result:
        if hasattr(part, "content") and isinstance(part.content, str):
            return part.content
    raise ValueError(f"No text content in MCP response: {result}")

raw_json = await fetch_events(plan_date)
print(f"Raw JSON:\n{raw_json[:500]}")

# Parse into our model
gcal_resp = GCalEventsResponse.model_validate_json(raw_json)
print(f"\n✅ Parsed {len(gcal_resp.events)} events for {plan_date}")
for ge in gcal_resp.events:
    start = ge.start.date_time or ge.start.date
    print(f"  • {ge.summary or '(no title)'} — {start}")

Raw JSON:
{"events":[],"totalCount":0}

✅ Parsed 0 events for 2026-02-08


In [14]:
"""
Live MCP test — Step 2: Full pipeline
  Create a TBPlan → generate MCP ops → submit to GCal → verify.
"""

# ── 1. Start from whatever GCal returned (0 events = empty baseline) ─────
baseline = gcal_response_to_tb_plan(gcal_resp, plan_date=plan_date, tz_name=TZ)
print(f"Baseline: {len(baseline.events)} events")

# ── 2. Simulate LLM output: a full day plan via ReplaceAll ───────────────
llm_patch = TBPatch(ops=[
    ReplaceAll(events=[
        TBEvent(n="Morning routine", d="Shower + coffee", t=ET.H,
                p=FixedStart(st=time(7, 0), dur=timedelta(minutes=30))),
        TBEvent(n="Deep work: thesis", d="Chapter 3 draft", t=ET.DW,
                p=AfterPrev(dur=timedelta(hours=2))),
        TBEvent(n="Coffee break", d="", t=ET.BU,
                p=AfterPrev(dur=timedelta(minutes=15))),
        TBEvent(n="Shallow work: emails", d="Inbox zero", t=ET.SW,
                p=AfterPrev(dur=timedelta(minutes=45))),
        TBEvent(n="Lunch", d="", t=ET.BU,
                p=AfterPrev(dur=timedelta(hours=1))),
        TBEvent(n="Deep work: thesis", d="Revisions", t=ET.DW,
                p=AfterPrev(dur=timedelta(hours=1, minutes=30))),
    ]),
])

# Apply patch
patched = apply_tb_ops(baseline, llm_patch)
resolved = patched.resolve_times()

print(f"\n=== Patched schedule ({len(resolved)} events) ===")
for r in resolved:
    print(f"  {r['start_time'].strftime('%H:%M')}–{r['end_time'].strftime('%H:%M')}  {r['n']} ({r['t']})")

# ── 3. Generate MCP ops (all creates since baseline was empty) ────────────
ops = tb_plan_to_mcp_ops(patched, calendar_id="primary")

print(f"\n=== {len(ops)} MCP operations ===")
for op in ops:
    verb = op.tool_name.replace("-event", "").upper()
    print(f"  {verb:8s} {op.arguments.get('summary', ''):30s} {op.arguments.get('start', '')}")

# ── 4. Submit to GCal via MCP ────────────────────────────────────────────
async def submit_ops(operations: list[MCPCalendarOp]) -> list[dict]:
    wb = McpWorkbench(StreamableHttpServerParams(url=MCP_URL, timeout=15))
    results = []
    async with wb:
        for op in operations:
            try:
                result = await wb.call_tool(op.tool_name, arguments=op.arguments)
                is_err = getattr(result, "is_error", False)
                # Extract text content from result
                text = ""
                for part in result.result:
                    if hasattr(part, "content"):
                        text = part.content
                        break
                results.append({
                    "tool": op.tool_name,
                    "summary": op.arguments.get("summary", ""),
                    "ok": not is_err,
                    "response": text[:200] if text else str(result)[:200],
                })
            except Exception as e:
                results.append({
                    "tool": op.tool_name,
                    "summary": op.arguments.get("summary", ""),
                    "ok": False,
                    "response": str(e)[:200],
                })
    return results

print("\n⏳ Submitting to GCal...")
submit_results = await submit_ops(ops)

ok_count = sum(1 for r in submit_results if r["ok"])
print(f"\n✅ {ok_count}/{len(submit_results)} operations succeeded")
for r in submit_results:
    status = "✅" if r["ok"] else "❌"
    print(f"  {status} {r['tool']:15s} {r['summary']:30s}")
    if not r["ok"]:
        print(f"     Error: {r['response']}")

# ── 5. Verify: fetch events again ────────────────────────────────────────
print("\n⏳ Verifying: re-fetching events from GCal...")
verify_json = await fetch_events(plan_date)
verify_resp = GCalEventsResponse.model_validate_json(verify_json)
print(f"\n✅ Verification: {len(verify_resp.events)} events on {plan_date}")
for ge in verify_resp.events:
    start = ge.start.date_time or ge.start.date
    print(f"  • {ge.summary or '(no title)'} — {start}")

Baseline: 0 events

=== Patched schedule (6 events) ===
  07:00–07:30  Morning routine (H)
  07:30–09:30  Deep work: thesis (DW)
  09:30–09:45  Coffee break (BU)
  09:45–10:30  Shallow work: emails (SW)
  10:30–11:30  Lunch (BU)
  11:30–13:00  Deep work: thesis (DW)

=== 6 MCP operations ===
  CREATE   Morning routine                2026-02-08T07:00:00+01:00
  CREATE   Deep work: thesis              2026-02-08T07:30:00+01:00
  CREATE   Coffee break                   2026-02-08T09:30:00+01:00
  CREATE   Shallow work: emails           2026-02-08T09:45:00+01:00
  CREATE   Lunch                          2026-02-08T10:30:00+01:00
  CREATE   Deep work: thesis              2026-02-08T11:30:00+01:00

⏳ Submitting to GCal...

✅ 6/6 operations succeeded
  ✅ create-event    Morning routine               
  ✅ create-event    Deep work: thesis             
  ✅ create-event    Coffee break                  
  ✅ create-event    Shallow work: emails          
  ✅ create-event    Lunch                 

In [16]:
"""
Live MCP test — Step 3: Patch existing events (update + delete + create).
Fetch back what we just created, apply edits, submit the diff.
"""

# ── 1. Fetch current events (the 6 we just created) ──────────────────────
current_json = await fetch_events(plan_date)
current_resp = GCalEventsResponse.model_validate_json(current_json)
print(f"📅 Current GCal events: {len(current_resp.events)}")

# Build baseline TBPlan from current GCal state
before = gcal_response_to_tb_plan(current_resp, plan_date=plan_date, tz_name=TZ)

# Build the GCal ID map: {tb_plan_index → gcal_event_id}
# Match by summary+start to handle duplicate summaries
gcal_id_lookup: dict[tuple[str, str], str] = {}
for ge in current_resp.events:
    if ge.start.date_time:
        gcal_id_lookup[(ge.summary or "", ge.start.date_time)] = ge.id

event_id_map: dict[int, str] = {}
for i, ev in enumerate(before.events):
    # before events are FixedWindow with concrete start
    st = ev.p.st if hasattr(ev.p, "st") else None
    if st:
        start_dt = datetime.combine(plan_date, st, tzinfo=ZoneInfo(TZ))
        key = (ev.n, start_dt.isoformat())
        if key in gcal_id_lookup:
            event_id_map[i] = gcal_id_lookup[key]

print(f"Event ID map ({len(event_id_map)} mapped):")
print(f"\nBefore ({len(before.events)} events):")
for i, ev in enumerate(before.events):
    gcal_id = event_id_map.get(i, "???")[:20]
    print(f"  [{i}] {ev.n:30s} ({ev.t.value}) — {ev.p}  gcal={gcal_id}")

# ── 2. Apply edits: rename, remove coffee break, add gym ─────────────────
# UpdateEvent uses: i (index), n/d/t/p (optional field updates)
edit_patch = TBPatch(ops=[
    # Rename "Shallow work: emails" → "Admin + Slack"
    UpdateEvent(i=3, n="Admin + Slack"),
    # Remove the coffee break (index 2)
    RemoveEvent(i=2),
    # Add a gym session at the end
    AddEvents(after=None, events=[
        TBEvent(n="Gym session", d="Legs day", t=ET.H,
                p=AfterPrev(dur=timedelta(hours=1))),
    ]),
])

after = apply_tb_ops(before, edit_patch)
after_resolved = after.resolve_times()

print(f"\nAfter ({len(after_resolved)} events):")
for r in after_resolved:
    print(f"  {r['start_time'].strftime('%H:%M')}–{r['end_time'].strftime('%H:%M')}  {r['n']} ({r['t']})")

# ── 3. Diff → minimal MCP ops ────────────────────────────────────────────
diff_ops = diff_tb_plans(before, after, event_id_map=event_id_map)

print(f"\n=== {len(diff_ops)} MCP diff operations ===")
for op in diff_ops:
    verb = op.tool_name.replace("-event", "").upper()
    summary = op.arguments.get("summary", "—")
    eid = op.arguments.get("eventId", "")[:20]
    print(f"  {verb:8s} {summary:30s} id={eid}")

# ── 4. Submit diff ───────────────────────────────────────────────────────
print("\n⏳ Submitting diff to GCal...")
diff_results = await submit_ops(diff_ops)

ok_count = sum(1 for r in diff_results if r["ok"])
print(f"\n✅ {ok_count}/{len(diff_results)} diff ops succeeded")
for r in diff_results:
    status = "✅" if r["ok"] else "❌"
    print(f"  {status} {r['tool']:15s} {r['summary']:30s}")
    if not r["ok"]:
        print(f"     Error: {r['response']}")

# ── 5. Final verification ────────────────────────────────────────────────
print("\n⏳ Final verification...")
final_json = await fetch_events(plan_date)
final_resp = GCalEventsResponse.model_validate_json(final_json)
print(f"\n✅ Final state: {len(final_resp.events)} events on {plan_date}")
for ge in final_resp.events:
    start = ge.start.date_time or ge.start.date
    print(f"  • {ge.summary or '(no title)'} — {start}")

📅 Current GCal events: 6
Event ID map (6 mapped):

Before (6 events):
  [0] Morning routine                (H) — a='fw' st=datetime.time(7, 0) et=datetime.time(7, 30)  gcal=fftbj73av1ltk1pmge3n
  [1] Deep work: thesis              (DW) — a='fw' st=datetime.time(7, 30) et=datetime.time(9, 30)  gcal=fftb9uqg08rcpmb55r1n
  [2] Coffee break                   (BU) — a='fw' st=datetime.time(9, 30) et=datetime.time(9, 45)  gcal=fftb02itvpp3vo75aopv
  [3] Shallow work: emails           (SW) — a='fw' st=datetime.time(9, 45) et=datetime.time(10, 30)  gcal=fftbn6kbqh87gm2dt0li
  [4] Lunch                          (BU) — a='fw' st=datetime.time(10, 30) et=datetime.time(11, 30)  gcal=fftbcsqf00rplk6rhe8t
  [5] Deep work: thesis              (DW) — a='fw' st=datetime.time(11, 30) et=datetime.time(13, 0)  gcal=fftbrm3t6ovsloulqpg7

After (6 events):
  07:00–07:30  Morning routine (H)
  07:30–09:30  Deep work: thesis (DW)
  09:45–10:30  Admin + Slack (SW)
  10:30–11:30  Lunch (BU)
  11:30–13:00  Deep 

In [17]:
"""
Live MCP test — Step 4: Test DELETE path.
Remove 2 events and verify they're deleted from GCal.
"""

# ── 1. Fetch current state ───────────────────────────────────────────────
current_json = await fetch_events(plan_date)
current_resp = GCalEventsResponse.model_validate_json(current_json)
print(f"📅 Current: {len(current_resp.events)} events")

before = gcal_response_to_tb_plan(current_resp, plan_date=plan_date, tz_name=TZ)

# Build event_id_map by summary+start
gcal_id_lookup: dict[tuple[str, str], str] = {}
for ge in current_resp.events:
    if ge.start.date_time:
        gcal_id_lookup[(ge.summary or "", ge.start.date_time)] = ge.id

event_id_map: dict[int, str] = {}
for i, ev in enumerate(before.events):
    st = ev.p.st if hasattr(ev.p, "st") else None
    if st:
        start_dt = datetime.combine(plan_date, st, tzinfo=ZoneInfo(TZ))
        key = (ev.n, start_dt.isoformat())
        if key in gcal_id_lookup:
            event_id_map[i] = gcal_id_lookup[key]

for i, ev in enumerate(before.events):
    gcal_id = event_id_map.get(i, "???")[:20]
    print(f"  [{i}] {ev.n:30s} gcal={gcal_id}")

# ── 2. Remove last 2 events ──────────────────────────────────────────────
n_before = len(before.events)
delete_patch = TBPatch(ops=[
    RemoveEvent(i=n_before - 1),  # remove last
    RemoveEvent(i=n_before - 2),  # remove second-to-last (after first removal, this is the new last)
])

after = apply_tb_ops(before, delete_patch)
print(f"\nAfter removal: {len(after.events)} events")
for r in after.resolve_times():
    print(f"  {r['start_time'].strftime('%H:%M')}–{r['end_time'].strftime('%H:%M')}  {r['n']}")

# ── 3. Diff → should produce 2 DELETEs ───────────────────────────────────
diff_ops = diff_tb_plans(before, after, event_id_map=event_id_map)
print(f"\n=== {len(diff_ops)} MCP diff ops ===")
for op in diff_ops:
    verb = op.tool_name.replace("-event", "").upper()
    summary = op.arguments.get("summary", "—")
    eid = op.arguments.get("eventId", "")[:20]
    print(f"  {verb:8s} {summary:30s} id={eid}")

# ── 4. Submit ─────────────────────────────────────────────────────────────
print("\n⏳ Submitting deletes...")
del_results = await submit_ops(diff_ops)
for r in del_results:
    status = "✅" if r["ok"] else "❌"
    print(f"  {status} {r['tool']:15s} {r['summary']:30s}")
    if not r["ok"]:
        print(f"     Error: {r['response']}")

# ── 5. Verify ─────────────────────────────────────────────────────────────
verify_json = await fetch_events(plan_date)
verify_resp = GCalEventsResponse.model_validate_json(verify_json)
print(f"\n✅ After delete: {len(verify_resp.events)} events (was {len(current_resp.events)})")
for ge in verify_resp.events:
    start = ge.start.date_time or ge.start.date
    print(f"  • {ge.summary or '(no title)'} — {start}")

📅 Current: 6 events
  [0] Morning routine                gcal=fftbj73av1ltk1pmge3n
  [1] Deep work: thesis              gcal=fftb9uqg08rcpmb55r1n
  [2] Admin + Slack                  gcal=fftb02itvpp3vo75aopv
  [3] Lunch                          gcal=fftbn6kbqh87gm2dt0li
  [4] Deep work: thesis              gcal=fftbcsqf00rplk6rhe8t
  [5] Gym session                    gcal=fftbrm3t6ovsloulqpg7

After removal: 4 events
  07:00–07:30  Morning routine
  07:30–09:30  Deep work: thesis
  09:45–10:30  Admin + Slack
  10:30–11:30  Lunch

=== 2 MCP diff ops ===
  DELETE   —                              id=fftbcsqf00rplk6rhe8t
  DELETE   —                              id=fftbrm3t6ovsloulqpg7

⏳ Submitting deletes...
  ✅ delete-event                                  
  ✅ delete-event                                  

✅ After delete: 4 events (was 6)
  • Morning routine — 2026-02-08T07:00:00+01:00
  • Deep work: thesis — 2026-02-08T07:30:00+01:00
  • Admin + Slack — 2026-02-08T09:45:00+01:00
  

In [18]:
"""
Cleanup: delete all remaining test events for plan_date.
"""
cleanup_json = await fetch_events(plan_date)
cleanup_resp = GCalEventsResponse.model_validate_json(cleanup_json)
print(f"🧹 Cleaning up {len(cleanup_resp.events)} events on {plan_date}...")

cleanup_ops = [
    MCPCalendarOp(
        tool_name="delete-event",
        arguments={"calendarId": "primary", "eventId": ge.id},
    )
    for ge in cleanup_resp.events
    if ge.id  # safety check
]

if cleanup_ops:
    results = await submit_ops(cleanup_ops)
    ok = sum(1 for r in results if r["ok"])
    print(f"✅ Deleted {ok}/{len(cleanup_ops)} events")
    for r in results:
        if not r["ok"]:
            print(f"  ❌ {r['response']}")
else:
    print("Nothing to clean up.")

# Verify empty
verify_json = await fetch_events(plan_date)
verify_resp = GCalEventsResponse.model_validate_json(verify_json)
print(f"\n📅 {plan_date}: {len(verify_resp.events)} events remaining")

🧹 Cleaning up 4 events on 2026-02-08...
✅ Deleted 4/4 events

📅 2026-02-08: 0 events remaining


In [8]:
from datetime import time, timedelta
from fateforger.agents.timeboxing.timebox import Timebox
from fateforger.agents.schedular.models.calendar import CalendarEvent, EventType

# Create proper validated objects then dump to dict (matching production MCP flow)
# Use model_construct to bypass SQLAlchemy instrumentation in notebook context
event = CalendarEvent.model_construct(
    summary="Deep work",
    event_type=EventType.DEEP_WORK,
    start_time=time(9, 0),
    duration=timedelta(hours=2),
)

initial_timebox = Timebox.model_construct(events=[event])

# Dump to dict, excluding computed fields that trigger SQLAlchemy
initial_timebox_dict = initial_timebox.model_dump(
    exclude={'events': {'__all__': {'colorId'}}}  # Exclude colorId computed field
)

model_client = build_autogen_chat_client("timebox_patcher", model=model)
timebox_patcher = ObjectPatcher(
    Timebox,
    model_client=model_client,
    initial_state=initial_timebox_dict,
    schema_by_alias=True,
)

timebox_instruction = (
    "Move the deep work to start at 10:00 and add a 30-minute buffer before it."
)

timebox_result = await timebox_patcher.run(timebox_instruction, max_attempts=3)
timebox_result

AttributeError: 'NoneType' object has no attribute 'supports_population'

In [ ]:
import toon
from typing import List, Literal
from pydantic import BaseModel, Field, field_validator, AliasChoices
import itertools
from openai import OpenAI

# 1. Global counter for IDs (The "Factory")
id_iterator = itertools.count(101)

class PizzaOrder(BaseModel):
    # ID is now optional in the constructor so Pydantic can handle it
    id: int = Field(default_factory=lambda: next(id_iterator))
    size: Literal["Small", "Medium", "Large"]
    crust: Literal["Thin", "Thick", "Stuffed"]
    toppings: List[str]

    @field_validator("toppings", mode="before")
    @classmethod
    def split_pipes(cls, v):
        if isinstance(v, str):
            return [t.strip() for t in v.split("|")]
        return v

class PizzaResponse(BaseModel):
    orders: List[PizzaOrder]

# 2. UPDATED GBNF: Notice 'id' is REMOVED from the row
# This forces the LLM to only provide the 3 data points we need.
pizza_gbnf = """
root      ::= header ":" "\\n" row+
header    ::= "orders[" [0-9]+ "]{size,crust,toppings}"
row       ::= "  " size "," crust "," toppings "\\n"

size      ::= "Small" | "Medium" | "Large"
crust     ::= "Thin" | "Thick" | "Stuffed"
toppings  ::= [a-zA-Z ]+ ("|" [a-zA-Z ]+)*
"""
# 2. Refined Prompt for proper TOON indentation
system_prompt = f"""You are a data generator.
Output strictly in TOON Tabular format.

{toon.generate_structure_from_pydantic(PizzaOrder)}

STRICT RULES:
1. Every row MUST be indented with exactly TWO SPACES.
2. Format: 
orders[count]{{size,crust,toppings}}:
  value,value,value
  value,value,value

3. No keys, no quotes, use '|' for toppings.
"""


response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Generate 3 random pizza orders."}
    ],
    extra_body={
        "guided_grammar": pizza_gbnf,
        "guided_decoding_backend": "xgrammar"
    }
)

raw_output = response.choices[0].message.content
print(f"--- Model Output ---\n{raw_output}")

# 4. Decode
# 4. Decode - FIXED: Use the wrapper class, then iterate through the .orders list
try:
    # This will now auto-assign IDs 101, 102, 103
    validated_response = toon.decode_to_pydantic(raw_output, PizzaResponse)
    
    for o in validated_response.orders:
        print(f"ID {o.id} (Auto): {o.size} {o.crust} - {o.toppings}")
except Exception as e:
    print(f"Parsing Failed: {e}")

--- Model Output ---
orders[3]{size,crust,toppings}:
  Large,Thin,Pepperoni|Mushrooms|Onions
  Medium,Stuffed,Sausage|Green Peppers
  Small,Gluten-Free,Spinach|Feta|Black Olives
Parsing Failed: 1 validation error for PizzaResponse
orders.2.crust
  Input should be 'Thin', 'Thick' or 'Stuffed' [type=literal_error, input_value='Gluten-Free', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error


In [ ]:
# Parse and validate the TOON string back to Pydantic objects
try:
    orders = decode_to_pydantic(toon_output, PizzaOrder)
    for order in orders:
        print(f"Validated Order: {order.id} - {order.size} {order.crust} pizza")
except Exception as e:
    print(f"Validation failed: {e}")

getting up at 9, its valentines day so we should do brunch, go to the market, light work after brunch, gym.
so i think lets order it like getting up, oats, mayve some light work, gym, brunch (around 11, 12?) and we need to do some groceries at the market, then get back and i want to do some work on ticketing and mapping c2f, and then valentines cooking dinner together around 18:00, dinner,

the one thing os to make a backlog for the C2F engine and the custom work for my client, and maybe try setting up some experiments or marketing and sales leads (like bart scheerder), o and i definitely have to respond to that professor guy.
 my DW blocks are usually 2 hours long..
so we have a max of two blocks (for this specific day, right?) with a small break in the middle, quesiton is how we are going to distribute that..